In [ ]:
import os 

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

azure_ai_project = os.environ.get("AZURE_PROJECT_ENDPOINT")
azure_openai_deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")
azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.environ.get("AZURE_OPENAI_API_KEY")  
azure_openai_api_version = os.environ.get("AZURE_OPENAI_API_VERSION")  

model_config = {
    "azure_endpoint": azure_openai_endpoint,
    "azure_deployment": azure_openai_deployment,
    "api_key": azure_openai_api_key,
}

In [ ]:
from azure.ai.evaluation import evaluate, GroundednessEvaluator, F1ScoreEvaluator, FluencyEvaluator
from model_endpoints import ModelEndpoints

groundedness_eval = GroundednessEvaluator(model_config=model_config)
f1_eval = F1ScoreEvaluator(threshold=0.6)
fluency_eval = FluencyEvaluator(model_config=model_config)

# Define a simple answer length evaluator
def answer_length(response, **kwargs):
    return {"answer_length": len(response)}

result = evaluate(
    data="hiking_data.jsonl", 
    target=ModelEndpoints("ministral"),
    evaluators={
        "groundedness": groundedness_eval,
        "answer_length": answer_length,
        "f1-score": f1_eval,
        "fluency": fluency_eval
    },
    
    evaluator_config={
        "groundedness": {
            "column_mapping": {
                "query": "${data.query}",
                "context": "${data.context}",
                "response": "${target.response}"
            } 
        }
    },
    
    azure_ai_project = azure_ai_project,
    output_path="./hiking-myevalresults.json"
)

In [ ]:
!pygmentize model_endpoints.py

In [ ]:
from model_endpoints import ModelEndpoints

ModelEndpoints(model_type="ministral").call_ministral_endpoint("What are some good hiking trails in California?")